# Authors: Andrea ZANON, Hamza ZERHOUNI

# Problems

## There are still infeasibilities

consider how big groups must be
could assume that we give teams a 1 week rest period... but that's not realistic

Andrea's idea:
optimize 4 weeks at a time for all teams

Hamza's improvement:
optimize 4 weeks at a time, for 4 groups with 5 teams in each group
first time pick random groups, from the second time solve an optimization problem to assign teams to groups under the constraint that tems that already played together cannot be on the same group
(are we sure that we are always able to find such groups? maybe we should create them at the same time initially to make sure they exist)

In [1]:
using DataFrames
using JuMP, Gurobi
using DataFrames

# Single league that scales

In [2]:
# number of teams
N = 20

# number of weeks
W = N-1
# number of days per weeks
D = 7; # we suppose week starts on tuesday, beacause monday is actually part of the previous football week


We define the matrix $C \in R^{TxT}$, with $C_{ij} = \frac{1}{j-i}$

In [6]:
function GetC()
    C = zeros((D*W, D*W))

    for t in 1:(W*D)
        for t_ in 1:(W*D)
            if t_ > t
                C[t,t_] = 1/sqrt(t_ - t)
            else
                C[t, t_] = 99999999999
            end
        end
    end

    return C
end

GetC (generic function with 1 method)

In [9]:
GetC();

Simple model, we are considering single league and making assumption that each team pla

In [10]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Monday=0.0, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0)
    for i=1:N
        for d=1:D
            for j=1:N
                if x[i,j,week,d] == 1.0
                    if d+2 == 9   
                        df[i, 2] = j
                    else
                        df[i,d+2] = j
                    end
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [42]:
function GamesWithinGroups(N_in, N_end, w_in, w_end, D)
    
    # create model
    model = Model(Gurobi.Optimizer)
    
    set_silent(model)

    # get C
    C = GetC()

    # VARIABLES

    # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, x[i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:D], Bin)
    @variable(model, a[i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:D], Bin)


    # OBJECTIVE

    # remember week begins on tuesday, this is why w_ goes from w+1
    @objective(model, Min, sum(C[7 * (w-1) + d, 7 * (w) + d_]* a[i, j, w, d] for i = N_in:N_end, 
        j = N_in:N_end, k = N_in:N_end, w = w_in:(w_end-1), d = 1:D, d_ = 1:D))

    # CONSTRAINTS

    # linearize objective function
    for i = N_in:N_end, j = N_in:N_end, k = N_in:N_end, w = w_in:(w_end-1), d = 1:D, d_ = 1:D
        @constraint(model, a[i,j,w,d] <= x[i,j,w,d])
        @constraint(model, a[i,j,w,d] <= x[i,k,w+1,d_])
        @constraint(model, a[i,j,w,d] >= x[i,j,w,d] + x[i,k,w+1,d_] - 1)
    end

    # each team can never play itself
    @constraint(model, [i = N_in:N_end, w = w_in:w_end, d = 1:D], x[i, i, w, d] <= 0)

    # if team A plays team B, then team B plays team A
    @constraint(model, [i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    # can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
    @constraint(model, [i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:3], x[i, j, w, d] <= 0)

    # each team plays at most one team per week
    @constraint(model, [i = N_in:N_end, j = N_in:N_end, w = w_in:w_end], sum(x[i, j, w, d] for d = 1:D) <= 1)

    # each team plays exactly N_end - N_in games
    @constraint(model, [i = N_in:N_end], sum(x[i, j, w, d] for j = N_in:N_end, w = w_in:w_end, d = 1:D) >= N_end - N_in)

    # all teams need to play each other exactly once
    for i = N_in:N_end
        for j = N_in:N_end
            if i != j
                @constraint(model, sum(x[i, j, w, d] for d = 1:D, w = w_in:w_end) == 1)
            end
        end
    end
    
    # solve model
    optimize!(model)

    return value.(x)
end

GamesWithinGroups (generic function with 1 method)

In [43]:
X = GamesWithinGroups(1, 5, 1, 4, 7);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17


In [23]:
for i = 1:5
    for j = 1:5
        for w = 1:4
            for d = 1:7
                if X[i, j, w, d] == 1.0
                    println("Team ", i, " plays team ", j, " on day ", d, " of week ", w)
                end
            end
        end
    end
end

Team 1 plays team 2 on day 4 of week 1
Team 1 plays team 3 on day 7 of week 1
Team 1 plays team 4 on day 4 of week 4
Team 1 plays team 5 on day 4 of week 1
Team 2 plays team 1 on day 4 of week 1
Team 2 plays team 3 on day 7 of week 1
Team 2 plays team 4 on day 4 of week 1
Team 2 plays team 5 on day 5 of week 1
Team 3 plays team 1 on day 7 of week 1
Team 3 plays team 2 on day 7 of week 1
Team 3 plays team 4 on day 4 of week 4
Team 3 plays team 5 on day 4 of week 1
Team 4 plays team 1 on day 4 of week 4
Team 4 plays team 2 on day 4 of week 1
Team 4 plays team 3 on day 4 of week 4
Team 4 plays team 5 on day 4 of week 1
Team 5 plays team 1 on day 4 of week 1
Team 5 plays team 2 on day 5 of week 1
Team 5 plays team 3 on day 4 of week 1
Team 5 plays team 4 on day 4 of week 1


In [17]:
# pass to functions: which teams i are within that group
function GamesBetweenGroups(i_in, i_end, j_in, j_end, w_in, w_end, D)

    # define model
    model = Model(Gurobi.Optimizer)

    # get C
    C = GetC()

    # take slicing of matrix C corresponding to the initial day and end day
    # c_in = 7 * (w_in - 1) + 1 # initial day (d = 1) of week w_in
    # c_end = 7 * (w_end - 1) + D # end day (d = D) of week w_end
    # C = C[c_in:c_end, c_in:c_end]

    # VARIABLES

    # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, x[i = i_in:i_end, j = j_in:j_end, w = w_in:w_end, d = 1:D], Bin)
    @variable(model, a[i = i_in:i_end, j = j_in:j_end, w = w_in:w_end, d = 1:D], Bin) # to linearize objective function

    # OBJECTIVE

    # remember week begins on tuesday, this is why w_ goes from w+1
    @objective(model, Min, sum(C[7 * (w-1) + d, 7 * w + d_]* a[i,j,w,d] for i in i_in:i_end, 
            j = j_in:j_end, k = j_in:j_end, w = w_in:(w_end-1), d = 1:D, d_ = 1:D))

    # CONSTRAINTS
    
    # linearize objective function
    for i = i_in:i_end, j = j_in:j_end, k = j_in:j_end, w = w_in:(w_end-1), d = 1:D, d_ = 1:D
        @constraint(model, a[i,j,w,d] <= x[i,j,w,d])
        @constraint(model, a[i,j,w,d] <= x[i,k,w+1,d_])
        @constraint(model, a[i,j,w,d] >= x[i,j,w,d] + x[i,k,w+1,d_] - 1)
    end
    
    # each team from one group must play exactly once against each team in the other group
    @constraint(model, [i = i_in:i_end, j = j_in:j_end], sum(x[i, j, w, d] for d = 1:D,w = w_in:w_end) == 1)

    # no two teams i can play against the same team j
    @constraint(model, [j = j_in:j_end, d = 1:D, w = w_in:w_end], sum(x[i, j, w, d] for i = i_in:i_end) <= 1)

    # can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
    @constraint(model, [i = i_in:i_end, j = j_in:j_end, w = w_in:w_end, d = 1:3], x[i, j, w, d] == 0)

    # at least four days of rest
    @constraint(model, [i = N_in:N_end, w = w_in:w_end], sum((7*w+d1)*x[i, j, w, d1] for j = N_in:N_end, d1 = 1:D) - sum((7*(w-1)+d2)x[i, k, w-1, d2] for k = N_in:N_end, d2 = 1:D) >= 4) # this makes it infeasible

    # solve model
    optimize!(model)
    
    return value.(x)

end

GamesBetweenGroups (generic function with 1 method)

In [ ]:
X = GamesBetweenGroups(1, 5, 6, 10, 1, 5, 7)